# Analisis de Datasets del TASS

In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
# Parse XML file
def parse_xml(filename):
    tree = ET.parse(filename)
    return tree.getroot()

In [3]:
def create_dataframe(filename):
    root = parse_xml(filename)
    columns = []
    datos = []
    
    for child in root:
        if not columns:
            columns = [element.tag for element in child]
        fila = [element.text for element in child]
        datos.append(fila)

    df = pd.DataFrame(datos, columns=columns)

    return df 

In [4]:
def create_sentiment_dataframe(filename):
    root = parse_xml(filename)
    tweets_data = []
    for tweet_elem in root.findall('tweet'):
        tweet_data = {}
        # Obtener elementos directos
        tweet_data['tweetid'] = tweet_elem.find('tweetid').text
        tweet_data['user'] = tweet_elem.find('user').text
        tweet_data['content'] = tweet_elem.find('content').text
        tweet_data['date'] = tweet_elem.find('date').text
        tweet_data['lang'] = tweet_elem.find('lang').text
        
        # Obtener elementos anidados sentiments
        sentiments_elem = tweet_elem.find('sentiments')
        if sentiments_elem is not None:
            tweet_data['sentiments'] = [{'entity': p.find('entity').text if p.find('entity') is not None else None,
                                'value': p.find('value').text if p.find('value') is not None else None,
                                'type': p.find('type').text if p.find('type') is not None else None}
                                for p in sentiments_elem.findall('polarity')]
        else:
            tweet_data['sentiments'] = []
        
        # Obtener elementos anidados topics
        topics_elem = tweet_elem.find('topics')
        if topics_elem is not None:
            tweet_data['topics'] = [t.text for t in topics_elem.findall('topic')]
        else:
            tweet_data['topics'] = []
        
        # Agregar datos del tweet a la lista
        tweets_data.append(tweet_data)
    return pd.DataFrame(tweets_data)

## 2012

Dataset de 158 usuarios con 15 columnas de metadata:
* Columns:
    * screenname
    * id 
    * name
    * description
    * url
    * profile_image_url
    * language
    * location 
    * timezone
    * followers_count
    * friends_count
    * created_at
    * type
    * twits
    * tendency : CENTER,RIGTH,LEFT, NONE

Datase de 7219 tweets con 7 columnas de metadatos:
* Incluye topicos
* Columns : ['tweetid', 'user', 'content', 'date', 'lang', 'sentiments', 'topics']
* Label de sentimiento Negativo, Neutro, Positivo , None
* Dentro del texto existen entidades a las que se les asignan el valor

In [8]:
users_data = create_dataframe("data/2012/general-users-tagged.xml")
users_data

,screenname,id,name,description,url,profile_image_url,lang,location,timezone,followers_count,friends_count,created_at,type,twits,tendency
0,jesusmarana,268470229,Jesús Maraña,Periodista. He sido director de 'Público' (y a...,http://www.jesusmarana.com,http://a0.twimg.com/profile_images/1284036321/...,es,Madrid,Madrid,24863,360,Fri Mar 18 21:02:44 +0000 2011,periodista,838,LEFT
1,EvaORegan,44903626,Evangeline O'Regan,"Journalist based in Madrid, Spain. She current...",http://www.intereconomia.com/blog/trinchera,http://a0.twimg.com/profile_images/1166247513/...,en,Madrid,None,1706,441,Fri Jun 05 14:30:19 +0000 2009,periodista,420,UNDEFINED
2,LosadaPescador,259799108,Luis Losada Pescador,Director del Telediario de Intereconomía de la...,http://www.intereconomia.com,http://a0.twimg.com/profile_images/1768407302/...,es,España,Madrid,3833,294,Wed Mar 02 16:33:49 +0000 2011,periodista,1492,CENTRE
3,pedroj_ramirez,263780425,Pedro J. Ramirez,Twitter oficial del Director de EL MUNDO,http://www.elmundo.es,http://a0.twimg.com/profile_images/1300084651/...,es,None,Madrid,95794,182,Thu Mar 10 18:39:39 +0000 2011,periodista,2454,CENTRE
4,SSantiagosegura,105451020,Santiago Segura,cosechando amiguetismo,None,http://a0.twimg.com/profile_images/1744151202/...,en,Madrid,None,984257,261,Sat Jan 16 11:20:59 +0000 2010,famoso,1706,UNDEFINED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,GUTY14HAZ,267274202,GUTI,twiiter oficial de jose maria gutierrez hernan...,None,http://a0.twimg.com/profile_images/1894888938/...,es,MADRID,Amsterdam,457008,16,Wed Mar 16 16:44:31 +0000 2011,famoso,161,RIGHT
154,gonzalezpons,106231214,González Pons,Diputado por Valencia y Vicesecretario General...,None,http://a0.twimg.com/profile_images/2078299767/...,es,None,Greenland,30215,569,Mon Jan 18 22:32:48 +0000 2010,político,197,RIGHT
155,manuelgomeztv,270281288,Manuel Gómez,Periodista. Presentador del Telediario 2 de I...,None,http://a0.twimg.com/profile_images/1903673010/...,es,España,None,669,140,Tue Mar 22 09:58:15 +0000 2011,periodista,7,RIGHT
156,BelenHoyo,239779545,Belén Hoyo Juliá,Diputada del @GPPopular. Miembro de la Junta D...,None,http://a0.twimg.com/profile_images/1896162840/...,es,Valencia,None,1752,859,Tue Jan 18 12:10:41 +0000 2011,político,108,RIGHT


In [9]:
df = create_sentiment_dataframe("data/2012/general-train-tagged-3l.xml")
df



,tweetid,user,content,date,lang,sentiments,topics
0,142389495503925248,ccifuentes,"Salgo de #VeoTV , que día más largoooooo...",2011-12-02T00:47:55,es,"[{'entity': None, 'value': 'NONE', 'type': 'AG...",[otros]
1,142389933619945473,CarmendelRiego,@PauladeLasHeras No te libraras de ayudar me/n...,2011-12-02T00:49:40,es,"[{'entity': None, 'value': 'NEU', 'type': 'DIS...",[otros]
2,142391947707940864,CarmendelRiego,@marodriguezb Gracias MAR,2011-12-02T00:57:40,es,"[{'entity': None, 'value': 'P', 'type': 'AGREE...",[otros]
3,142416095012339712,mgilguerrero,"Off pensando en el regalito Sinde, la que se v...",2011-12-02T02:33:37,es,"[{'entity': None, 'value': 'N', 'type': 'AGREE...","[política, economía]"
4,142422495721562112,paurubio,Conozco a alguien q es adicto al drama! Ja ja ...,2011-12-02T02:59:03,es,"[{'entity': None, 'value': 'P', 'type': 'AGREE...",[otros]
...,...,...,...,...,...,...,...
7214,189809904935120896,mariviromero,Muy indignante si ...nadie repara en ello hoy ...,2012-04-10T22:19:42,es,"[{'entity': None, 'value': 'N', 'type': 'AGREE...",[entretenimiento]
7215,189814854503235584,mariviromero,Más pobres por discriminar a la mujer http://t...,2012-04-10T22:39:22,es,"[{'entity': None, 'value': 'N', 'type': 'AGREE...",[otros]
7216,189815389021143041,mariviromero,Crean un banco de productos para mujeres con c...,2012-04-10T22:41:30,es,"[{'entity': None, 'value': 'P', 'type': 'AGREE...",[tecnología]
7217,189824278827507712,Tonicanto1,Sobre la sorprendente huída hoy en el Senado d...,2012-04-10T23:16:49,es,"[{'entity': None, 'value': 'N', 'type': 'AGREE...",[política]


In [28]:
df_topicos= df.explode('topics')

conteo_topicos = df_topicos['topics'].value_counts()

# Mostrar el resultado
print(conteo_topicos)

topics
política           3120
otros              2337
entretenimiento    1678
economía            942
música              566
fútbol              252
cine                245
tecnología          217
deportes            113
literatura          103
Name: count, dtype: int64


## 2013

Dataset de politica, 2500 tweets con 7 columnas
* Columnas: ['tweetid', 'user', 'content', 'date', 'lang', 'sentiments', 'topics']
* Label de sentimiento Negativo, Neutro, Positivo; Ninguno
* Dentro del texto existen entidades a las que se les asignan el valor de sentimiento

In [10]:
politics_df = create_sentiment_dataframe("data/2013/politics-test-tagged.xml")
politics_df

,tweetid,user,content,date,lang,sentiments,topics
0,137228516625367040,TonyKrdniosa,"""@marianorajoy: En España las cosas se pueden,...",2011-10-17T19:00:02,es,"[{'entity': None, 'value': 'P', 'type': 'AGREE...",[política]
1,137228522019229697,elhijodelapepa,"En PSO€ el que no corre vuela, todavía calient...",2011-10-17T19:00:03,es,"[{'entity': None, 'value': 'N', 'type': 'DISAG...",[política]
2,137228533029277696,MarisaVargas_R,#nomeolvido de q cuando se aprobo la ley del a...,2011-10-17T19:00:06,es,"[{'entity': None, 'value': 'N', 'type': 'AGREE...",[política]
3,137228551198998528,Javier_Diego,#CCOO exige al nuevo Gobierno que reactive el ...,2011-10-17T19:00:10,es,"[{'entity': None, 'value': 'NEU', 'type': 'AGR...",[política]
4,137228569750405120,Alma_Cantaro,@marianorajoy A mí inviable me parecen los fra...,2011-10-17T19:00:15,es,"[{'entity': None, 'value': 'P', 'type': 'DISAG...",[política]
...,...,...,...,...,...,...,...
2495,137523161032953856,anabusta,"#rebélate @iunida @cayo_lara ""¿Dinero a los ba...",2011-10-18T14:30:51,es,"[{'entity': None, 'value': 'N', 'type': 'AGREE...",[política]
2496,137523199150796800,europapress_es,". @cayo_lara: ""La prima de riesgo es la niña d...",2011-10-18T14:31:00,es,"[{'entity': None, 'value': 'NEU', 'type': 'DIS...",[política]
2497,137523450393788416,CatalanGarcia1,@cayo_lara Vuestra propuesta decreación de emp...,2011-10-18T14:32:00,es,"[{'entity': None, 'value': 'P', 'type': 'AGREE...",[política]
2498,137523567108685824,anabusta,"#rebelate @iunida cayo_lara: ""La prima de ries...",2011-10-18T14:32:28,es,"[{'entity': None, 'value': 'NEU', 'type': 'DIS...",[política]


In [30]:
politics_topics= politics_df.explode('topics')

cant_politic_topics = politics_topics['topics'].value_counts()

print(cant_politic_topics )

topics
política    2500
Name: count, dtype: int64


## 2014

Dataset de entretenimiento de 1773 filas
 * Columnas : ['id', 'texto', 'sentimiento']
 * Sentimiento: 
   - [{'aspect': 'Afición', 'polarity': 'P'},
   - {'aspect': 'Equipo-Real_Madrid', 'polarity': 'P'},
   - {'aspect': 'Equipo', 'polarity': 'P'}]

In [11]:
def crate_social_dataframe(filename):
    root = parse_xml(filename)
    df_columns = ['id', 'texto', 'sentimiento']
    data = {'id': [], 'texto': [], 'sentimiento': []}
    for tweet_elem in root.findall('tweet'):
        tweet_data = {}
        tweet_data['id'] = tweet_elem.get('id')
        
        sentimientos = []
        for sentiment_elem in tweet_elem.findall('sentiment'):
            aspect = sentiment_elem.get('aspect')
            polarity = sentiment_elem.get('polarity')
            sentimientos.append({'aspect': aspect, 'polarity': polarity})
        tweet_data['sentimiento'] = sentimientos
        
        
        texto_completo = ''
        for elem in tweet_elem.itertext():
            texto_completo += elem.strip()
            texto_completo += " "
        tweet_data['texto'] = texto_completo if texto_completo else None


        for col in df_columns:
            data[col].append(tweet_data[col])
    
    return pd.DataFrame(data)    

In [12]:
social_train = crate_social_dataframe("data/2014/socialtv-train-tagged.xml") 
social_train

,id,texto,sentimiento
0,456544889786728451,Los que esta noche van a la redonda a celebrar...,"[{'aspect': 'Afición', 'polarity': 'P'}, {'asp..."
1,456544890004852736,Diooooos que careron de gareth madreeeeee ahí ...,"[{'aspect': 'Jugador-Gareth_Bale', 'polarity':..."
2,456544890231353345,Ganó el mejor. @realmadrid #CopaRelRey #Fina...,"[{'aspect': 'Equipo-Real_Madrid', 'polarity': ..."
3,456544890336206848,@realmadrid Felicidades!!! ? Gracias por alegr...,"[{'aspect': 'Equipo-Real_Madrid', 'polarity': ..."
4,456544890533339136,@titelas Mañana Bale en naranja,"[{'aspect': 'Jugador-Gareth_Bale', 'polarity':..."
...,...,...,...
1768,456544991775436801,Hahahaha la cara de messi csm jajajajjaja,"[{'aspect': 'Jugador-Lionel_Messi', 'polarity'..."
1769,456544992119361536,"""Mi novia @agudavoice quiere que la ame, pero...","[{'aspect': 'Jugador-Gareth_Bale', 'polarity':..."
1770,456544994996678656,Foto | ¿Hubo agresión de Pepe a Neymar en la f...,"[{'aspect': 'Retransmisión', 'polarity': 'NEU'..."
1771,456544995005054976,Pinto debería porterear con sotana... http://t...,"[{'aspect': 'Jugador-José_Manuel_Pinto', 'pola..."


## 2015

Dataset de politica de 784 filas
 * Columnas : ['id', 'texto', 'sentimiento']
 * Sentimiento: [{'aspect': 'Economia', 'polarity': 'N'}]

In [13]:
stompol_train = crate_social_dataframe("data/2015/stompol-train-tagged.xml")
stompol_train

,id,texto,sentimiento
0,591154373696323584,Diga cuanto nos van a costar las autovías de s...,"[{'aspect': 'Economia', 'polarity': 'N'}]"
1,591154532362670080,@lhermoso_ @sanchezcastejon #DobleMoral Castil...,"[{'aspect': 'Propio_partido', 'polarity': 'N'}]"
2,591154730111500288,@PSOE @sanchezcastejon Me hace mucha gracia qu...,"[{'aspect': 'Propio_partido', 'polarity': 'N'}]"
3,591154874131308544,Para que todo el mundo lo tengo claro .... @Cs...,"[{'aspect': 'Propio_partido', 'polarity': 'NEU'}]"
4,591154909384409088,@Albert_Rivera y @CiudadanosCs saben q este pa...,"[{'aspect': 'Propio_partido', 'polarity': 'N'}..."
...,...,...,...
779,591482509013028864,@cs_hor @AntSoubrieCs @Albert_Rivera @Cs_Alcob...,"[{'aspect': 'Economia', 'polarity': 'N'}]"
780,591483818088923137,El PP ganaría la Comunidad de Madrid y el Ayun...,"[{'aspect': 'Propio_partido', 'polarity': 'NEU..."
781,591484734770843650,¿Quién es Pedro Sánchez ? Mi artículo de los v...,"[{'aspect': 'Propio_partido', 'polarity': 'NEU..."
782,591485374783860736,Hoy mucha propaganda del IBEX-35 en las encues...,"[{'aspect': 'Propio_partido', 'polarity': 'P'}..."


## 2019



Datasets con tweets con léxico propio de diversos paises:
 * Columnas :  ['tweetid', 'user', 'content', 'date', 'lang', 'sentiment_value']
 * Sentiment value = None, Negative, Positive

Tweets por paises: 
 
| Pais     | Train | Dev     | **Total**   |
|----------|-------|---------|-------- |
| Costa Rica | 777   | 390   | 1167    |
| España     | 1008  | 506   | 1514    |
| Uruguay    | 943   | 486   | 1429    |
| Mexico     | 989   | 510   | 1499    |
| Peru       | 966   | 498   | 1464    |
| **Total**      | **4683**  | **2390**  | **7073**    |


In [14]:
def create_inter_dataframe(filename):
    root = parse_xml(filename)
    tweets_data = []
    for tweet_elem in root.findall('tweet'):
        tweet_data = {}
        
        tweet_data['tweetid'] = tweet_elem.find('tweetid').text
        tweet_data['user'] = tweet_elem.find('user').text
        tweet_data['content'] = tweet_elem.find('content').text
        tweet_data['date'] = tweet_elem.find('date').text
        tweet_data['lang'] = tweet_elem.find('lang').text
        
        sentiment_elem = tweet_elem.find('sentiment')
        
        polarity_elem = sentiment_elem.find('polarity')
        
        tweet_data['sentiment_value'] = polarity_elem.find('value').text if polarity_elem is not None else None
        
        tweets_data.append(tweet_data)


    return pd.DataFrame(tweets_data)

### Costa Rica

In [15]:
cr_train = create_inter_dataframe("data/2019/TASS2019_country_CR_train.xml")
cr_train


,tweetid,user,content,date,lang,sentiment_value
0,768225400254111744,14628107,@NoilyMV yo soy totalmente puntual,Tue Aug 23 23:16:23 +0000 2016,es,NONE
1,770077064833671168,713600676,@SandraCauffman Hola Sandrita. No le habia des...,Mon Aug 29 01:54:14 +0000 2016,es,P
2,771207534342320128,120940293,Si andan haciendo eso mejor se quedaran callad...,Thu Sep 01 04:46:19 +0000 2016,es,N
3,771900763987513345,2827444381,Que pereza quiero choco banano,Sat Sep 03 02:40:58 +0000 2016,es,N
4,772550560998301697,60878511,"@robertobrenes Bueno, no es tanto lo mayor com...",Sun Sep 04 21:43:01 +0000 2016,es,N
...,...,...,...,...,...,...
772,814853214050906112,311102472,Quiero como tener carro para hacer todo rápido,Fri Dec 30 15:18:40 +0000 2016,es,NEU
773,815439552038977537,119492787,Tristeza más hp me da siempre en año nuevo,Sun Jan 01 06:08:33 +0000 2017,es,N
774,817864316372348928,4878793919,no encuentro ninguna buena serie en NF.,Sat Jan 07 22:43:42 +0000 2017,es,N
775,817069708927266816,722185474182254598,@Alphasniper97 Alpha suerte y ten un feliz año...,Thu Jan 05 18:06:13 +0000 2017,es,P


In [16]:
cr_dev = create_inter_dataframe("data/2019/TASS2019_country_CR_dev.xml")
cr_dev

,tweetid,user,content,date,lang,sentiment_value
0,769802080701648896,2774677761,@DexterAOM por eso es que se dice lo del final...,Sun Aug 28 07:41:32 +0000 2016,es,NEU
1,772253053328105472,713600676,"@kikorobles11 6 meses con la banda y largo, fu...",Sun Sep 04 02:00:50 +0000 2016,es,NONE
2,775091274475003904,104178963,@bichoclarke @AndreaVasRo El otro día me quejé...,Sun Sep 11 21:58:54 +0000 2016,es,N
3,768874738089275393,627838882,El sábado se casa Adriel y eso me hace sentir ...,Thu Aug 25 18:16:37 +0000 2016,es,NEU
4,767564596177666048,90973470,@ferrg25 igual uno ya está en el baúl de los r...,Mon Aug 22 03:30:35 +0000 2016,es,NEU
...,...,...,...,...,...,...
385,819406610636357632,147745734,"@leonora_monge @susi_sot por mi sí, con 20 rojos",Thu Jan 12 04:52:14 +0000 2017,es,NONE
386,819337926026862593,1352062208,Antojo de empanada colombiana,Thu Jan 12 00:19:18 +0000 2017,es,NEU
387,819383109334552576,65718560,@mendezcr nah! Yo estoy lleno de tattos y me a...,Thu Jan 12 03:18:51 +0000 2017,es,P
388,819366776052060161,223818187,"@Mayel_Oh Cuando te lo creés, lo sos. Pero bue...",Thu Jan 12 02:13:57 +0000 2017,es,N


### España

In [17]:
es_train = create_inter_dataframe("data/2019/intertass-ES-train-tagged.xml")
es_train

,tweetid,user,content,date,lang,sentiment_value
0,768213876278165504,OnceBukowski,-Me caes muy bien \n-Tienes que jugar más part...,2016-08-23 22:30:35,es,NONE
1,768213567418036224,anahorxn,@myendlesshazza a. que puto mal escribo\n\nb. ...,2016-08-23 22:29:21,es,N
2,768212591105703936,martitarey13,@estherct209 jajajaja la tuya y la d mucha gen...,2016-08-23 22:25:29,es,N
3,768221670255493120,endlessmilerr,Quiero mogollón a @AlbaBenito99 pero sobretodo...,2016-08-23 23:01:33,es,P
4,768221021300264964,JunoWTFL,Vale he visto la tia bebiendose su regla y me ...,2016-08-23 22:58:58,es,N
...,...,...,...,...,...,...
1003,814846333601320960,jdomperfects,Para mí mi mejor año fue el 2015,2016-12-30 14:51:19,es,P
1004,813731371076243461,GonzaloRoman_91,Hoy va a ser un dia muy largo...,2016-12-27 13:00:51,es,N
1005,818399956792905728,vaneftdallas,11:11 que no me encuentre mal,2017-01-09 10:12:09,es,NEU
1006,815715581878009858,carmen_ligero,Es que son muy bonitas jo,2017-01-02 00:25:24,es,P


In [18]:
es_dev = create_inter_dataframe("data/2019/intertass-ES-development-tagged.xml")
es_dev

,tweetid,user,content,date,lang,sentiment_value
0,770976639173951488,noseashetero,@noseashetero 1000/10 de verdad a ti que voy a...,2016-08-31 13:28:49,es,P
1,771092421866389508,Templelx,@piscolabisaereo @HistoriaNG @SPosteguillo las...,2016-08-31 21:08:54,es,P
2,771092111429083136,esskuu94,"Al final han sido 3h Bueno, mañana tengo fies...",2016-08-31 21:07:40,es,P
3,771092070572449796,__ariadna9,@Jorge_Ruiz14 yo no tengo tiempo para esas cos...,2016-08-31 21:07:30,es,N
4,771094192508600320,_cristtina15_,@_MissChaotic_ ves ese brillo? es un coso que ...,2016-08-31 21:15:56,es,N
...,...,...,...,...,...,...
501,757593585634213888,RubenRuCh,@Davorias Pero es que está muy rica,2016-07-25 15:09:20,es,P
502,757585725764362240,rayjaken,@narvalillo Eso es antiguo ya,2016-07-25 14:38:06,es,N
503,757097408304148480,Fuhyo__,Mi TL esta vacía a estas horas,2016-07-24 06:17:42,es,N
504,757971881077182464,xLytherinDovah,"Ya tengo uno, pero ese es más cheto @aweamasome.",2016-07-26 16:12:33,es,NEU


### Perú

In [19]:
pe_train = create_inter_dataframe("data/2019/TASS2019_country_PE_train.xml")
pe_train


,tweetid,user,content,date,lang,sentiment_value
0,772432598027145216,71546415,"Sin ser fan de Juan Gabriel, siempre supe que ...",Sun Sep 04 13:54:17 +0000 2016,es,P
1,772489016352669701,53311422,Sabes que no tendrás un buen día cuando lo pri...,Sun Sep 04 17:38:28 +0000 2016,es,N
2,771317218634149888,599653674,"En situaciones en las que no sepas que hacer, ...",Thu Sep 01 12:02:09 +0000 2016,es,N
3,771715645843079169,106919551,ayer preguntaban y dónde están las solteras!!!...,Fri Sep 02 14:25:22 +0000 2016,es,NEU
4,771744506861461504,599653674,Que el finde sea para hacer cualquier cosa que...,Fri Sep 02 16:20:03 +0000 2016,es,NEU
...,...,...,...,...,...,...
961,818130147366813696,105705498,"Terminarán odiándome, como casi todos los padr...",Sun Jan 08 16:20:01 +0000 2017,es,N
962,816876533231079424,1005307664,Necesito urgente entrar a la gimnasia y al gim...,Thu Jan 05 05:18:37 +0000 2017,es,NONE
963,816890935808323584,112088294,Lo que me recuerda que @edvergaray me criticar...,Thu Jan 05 06:15:50 +0000 2017,es,NONE
964,818483269255761920,711592506329329666,"Hasta lo peor de mi, quiere lo mejor para ti",Mon Jan 09 15:43:12 +0000 2017,es,NEU


In [20]:
pe_dev = create_inter_dataframe("data/2019/TASS2019_country_PE_dev.xml")
pe_dev

,tweetid,user,content,date,lang,sentiment_value
0,769672659793801216,560455070,Así te paguen bien... Si es a última hora... N...,Sat Aug 27 23:07:16 +0000 2016,es,N
1,770775991090380800,76040253,Manolo: se llama H&M por Hombre y Mujer. Yo: ....,Wed Aug 31 00:11:31 +0000 2016,es,NONE
2,775046786134241280,50862843,Buen resumen de mi vida amorosa. - Lo he amado...,Sun Sep 11 19:02:08 +0000 2016,es,N
3,768889490383446016,44719182,@BartanSoo12 Pero tú ya sabes por qué le digo ...,Thu Aug 25 19:15:14 +0000 2016,es,NONE
4,768333118239412224,714677089589452804,Viendo con mi gordiflaqui LOCA POR LAS COMPRAS...,Wed Aug 24 06:24:24 +0000 2016,es,NONE
...,...,...,...,...,...,...
493,819397621106221057,346951471,@JohnsitoCb Ya sabía que un chivito se iría a ...,Thu Jan 12 04:16:31 +0000 2017,es,NONE
494,819277149274275840,277318489,Hazme un favor y has lo posible por no cruzart...,Wed Jan 11 20:17:48 +0000 2017,es,NONE
495,819276021988020230,324579655,Bueeeeno... no me siento bien. Gracias playlis...,Wed Jan 11 20:13:19 +0000 2017,es,N
496,819423188580966400,3423300857,D.O eres genial ...Feliz cumpleaños y q sean m...,Thu Jan 12 05:58:06 +0000 2017,es,P


### Uruguay

In [21]:
ur_train = create_inter_dataframe("data/2019/TASS2019_country_UY_train.xml")
ur_train

,tweetid,user,content,date,lang,sentiment_value
0,768529421548609536,293627746,Yo espero tranquila. Si gana de nvo alg partid...,Wed Aug 24 19:24:27 +0000 2016,es,NEU
1,771028255629512706,35709180,"Los fracasos de Michelle, Cristina y Dilma mue...",Wed Aug 31 16:53:55 +0000 2016,es,N
2,771055632854183940,146577758,Debemos orar por los hnos venezolanos gente q ...,Wed Aug 31 18:42:43 +0000 2016,es,N
3,770138191689093122,355883744,Me dijo que no le había gustado no se cuantas ...,Mon Aug 29 05:57:08 +0000 2016,es,NEU
4,769716104491040768,2701106364,@susanagkp jajaja yo fui el viernes ! precavid...,Sun Aug 28 01:59:54 +0000 2016,es,N
...,...,...,...,...,...,...
938,811436261059792896,2227486286,@AmorAKilates @Roocio_Mk si me pasa lo mismo!,Wed Dec 21 05:00:55 +0000 2016,es,NONE
939,809280785182322688,140260506,@clauchoarrionda pquno ladra y ls demas retwitean,Thu Dec 15 06:15:49 +0000 2016,es,NEU
940,810772298911350784,3291225459,A mi desayuno le hizo falta un alfajor podrida...,Mon Dec 19 09:02:34 +0000 2016,es,N
941,813200690894045184,2257050336,Viste cuando necesitas que alguien te escuche ...,Mon Dec 26 01:52:07 +0000 2016,es,N


In [22]:
ur_dev = create_inter_dataframe("data/2019/TASS2019_country_UY_dev.xml")
ur_dev

,tweetid,user,content,date,lang,sentiment_value
0,768113609725489152,707984087085129729,@cuervotinelli estoy preocupado !! ... Me pon...,Tue Aug 23 15:52:10 +0000 2016,es,NEU
1,768862180104138752,1582261890,hoy en musimundo van a estar filmando discos d...,Thu Aug 25 17:26:43 +0000 2016,es,N
2,767352207146713088,360134852,"@osilveira buena! ""Cría cuervos, que te comerá...",Sun Aug 21 13:26:37 +0000 2016,es,N
3,769233349651099648,2701106364,@Adri_tranqui como en el caso de invitar a alg...,Fri Aug 26 18:01:36 +0000 2016,es,NEU
4,769477038021439489,4323201982,No se tomen nunca el copa de las 6:15 en Mtvo ...,Sat Aug 27 10:09:56 +0000 2016,es,N
...,...,...,...,...,...,...
481,819404084637790208,239060004,@sonmifelicidad_ yo el programa!! sino lo veo ...,Thu Jan 12 04:42:12 +0000 2017,es,N
482,819405921000583171,239060004,@sonmifelicidad_ igual este Lunes seguro vea L...,Thu Jan 12 04:49:30 +0000 2017,es,NEU
483,819398651759955968,4623948849,@Guille2SantosTM @urbandoficial Al contrario g...,Thu Jan 12 04:20:36 +0000 2017,es,P
484,819397460908974080,284595550,@juliapresa0 todas julinator te quiero y extraño,Thu Jan 12 04:15:53 +0000 2017,es,P


### México

In [23]:
mx_train = create_inter_dataframe("data/2019/TASS2019_country_MX_train.xml")
mx_train


,tweetid,user,content,date,lang,sentiment_value
0,767817507897872385,71463624,— ¿Nivel de inglés? — Alto. — Traduzca juguete...,Mon Aug 22 20:15:33 +0000 2016,es,NEU
1,772486813948575745,572924309,Si soy de área 4 y con suerte pase mate IV y m...,Sun Sep 04 17:29:43 +0000 2016,es,N
2,771871884451295233,158142748,Sabían que su after fav Teatro G. fue construi...,Sat Sep 03 00:46:12 +0000 2016,es,P
3,771947976302456832,132254421,Y hoy por primera vez me salí con el permiso d...,Sat Sep 03 05:48:34 +0000 2016,es,P
4,774741645484896256,105530236,"¿Para qué hacen coraje con Ambriz?, al final n...",Sat Sep 10 22:49:36 +0000 2016,es,N
...,...,...,...,...,...,...
984,818773396318277633,195469214,Si quieren que aprenda algo de eso dejen esa m...,Tue Jan 10 10:56:04 +0000 2017,es,NEU
985,818738784657408000,93545559,Últimos momentos en medicina familiar y de gua...,Tue Jan 10 08:38:32 +0000 2017,es,NONE
986,819050320177152000,322339981,@natyamezcua hay pues hay que ver que hacemos ...,Wed Jan 11 05:16:28 +0000 2017,es,NEU
987,819049335602216962,627591478,Me acabo de tragar un pastel en mood celina,Wed Jan 11 05:12:33 +0000 2017,es,N


In [24]:
mx_dev = create_inter_dataframe("data/2019/TASS2019_country_MX_dev.xml")
mx_dev

,tweetid,user,content,date,lang,sentiment_value
0,774677024673714176,288849596,Hoy le dí asesoría a 1 dude q qería recuperar ...,Sat Sep 10 18:32:50 +0000 2016,es,P
1,773710630956130304,610531063,¡Rápido a qué comprar el iPhone 7 antes que sa...,Thu Sep 08 02:32:43 +0000 2016,es,NEU
2,764625478317596672,188013030,Sólo y triste bajo el sol en la playa busco am...,Sun Aug 14 00:51:34 +0000 2016,es,NEU
3,756002242503643141,583337287,Contestarle a mi novio 2hrs y media después po...,Thu Jul 21 05:45:55 +0000 2016,es,N
4,779494837418463232,736673890194313217,Quien se va de party hoyyy.?? Yooo..!!! Quien ...,Sat Sep 24 01:37:06 +0000 2016,es,P
...,...,...,...,...,...,...
505,819365162302013443,125172426,"tomando en miércoles?... na, sólo olvide mis l...",Thu Jan 12 02:07:32 +0000 2017,es,N
506,819456610829471744,2261362278,quiero y necesito verte yaaaa,Thu Jan 12 08:10:55 +0000 2017,es,P
507,819456529543798784,261336416,Gracias a Comedy Central y MTV voy a estar en ...,Thu Jan 12 08:10:36 +0000 2017,es,P
508,819469945167720448,17614100,@andresg9108 y que los demas solo den npm inst...,Thu Jan 12 09:03:54 +0000 2017,es,NONE


## 2020

Dataset en español con 5886 (train) + 857 (dev) tweets con emociones:
   - anger (also includes annoyance and rage) 
   - disgust (also includes disinterest, dislike and loathing)
   - fear (also includes apprehension, anxiety, concern, and terror) 
   - joy (also includes serenity and ecstasy) 
   - sadness (also includes pensiveness and grief) 
   - surprise (also includes distraction and amazement) 
Columns: ['id', 'tweet', 'label ']
Shap

In [7]:
emotions_train = pd.read_csv("data/2020/train.tsv", sep='\t')
emotions_train


,id,tweet,label
0,1,El Atlético resignado a perder HASHTAG 😔 http...,sadness
1,2,Leer proporciona una mejor visión del mundo 🤓 ...,joy
2,3,Amo a Arya Stark por encima de todas las cosas...,joy
3,4,Gracias HASHTAG es increíble que una niña logr...,others
4,5,Solo siento que hayamos perdido 24 escaños de ...,sadness
...,...,...,...
5881,5882,Pequeños gestos que puedes hacer por el medio ...,others
5882,5883,¿Sabías que de las dos coronaciones que se cel...,surprise
5883,5884,Mientras reflexionamos sobre lo duro que es es...,anger
5884,5885,La fachada de la catedral de Notre Dame fue “s...,sadness


In [8]:
emotions_dev = pd.read_csv("data/2020/dev.tsv", sep='\t')
emotions_dev

,id,tweet,label
0,1,-Spoiler- Nadie ha tenido el dilema de tener q...,others
1,2,Yo solo quiero saber donde esta HASHTAG? HASHT...,others
2,3,"La literatura nos hace más empáticos, dispuest...",joy
3,4,"Para mi, estas son las 4 mejores escenas de es...",surprise
4,5,Lo que acabo de ver es puro oro. Historia de l...,joy
...,...,...,...
852,853,"USER ya no te duermas con lentes puestos, si n...",others
853,854,"En el HASHTAG, recordamos las mejores recomend...",others
854,855,¡Feliz HASHTAG! 📚 Esperamos que os hayan regal...,joy
855,856,Que lo mismo l@s español@s no somos imbéciles ...,others


In [10]:
df_union = pd.concat([emotions_dev, emotions_train], ignore_index=True)
df_union

,id,tweet,label
0,1,-Spoiler- Nadie ha tenido el dilema de tener q...,others
1,2,Yo solo quiero saber donde esta HASHTAG? HASHT...,others
2,3,"La literatura nos hace más empáticos, dispuest...",joy
3,4,"Para mi, estas son las 4 mejores escenas de es...",surprise
4,5,Lo que acabo de ver es puro oro. Historia de l...,joy
...,...,...,...
6738,5882,Pequeños gestos que puedes hacer por el medio ...,others
6739,5883,¿Sabías que de las dos coronaciones que se cel...,surprise
6740,5884,Mientras reflexionamos sobre lo duro que es es...,anger
6741,5885,La fachada de la catedral de Notre Dame fue “s...,sadness


In [18]:
print(df_union.columns)

Index(['id', 'tweet', 'label '], dtype='object')


In [24]:
df_union["label "].value_counts()

label 
others       3310
joy          1455
sadness       809
anger         687
surprise      276
disgust       129
fear           77
Name: count, dtype: int64